In [2]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Consignment-Pricing-Prediction'

In [69]:
import pandas as pd

df = pd.read_csv("artifacts\data_ingestion\SCMS_Delivery_History_Dataset.csv", usecols=[
    'Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)', 'Line Item Insurance (USD)','Scheduled Delivery Date', 'Delivered to Client Date'])

# df = pd.read_csv('artifacts/data_ingestion/SCMS_Delivery_History_Dataset.csv', index_col=['ID', 'PO / SO #', 'ASN/DN #', 'Managed By', 'PO Sent to Vendor Date',
#        'First Line Designation'])

In [70]:
df.columns 

Index(['Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [71]:
df.head()

,Project Code,PQ #,Country,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,...,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,100-CI-T01,Pre-PQ Process,Côte d'Ivoire,Direct Drop,EXW,Air,Pre-PQ Process,2-Jun-06,2-Jun-06,2-Jun-06,...,Test kit,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,13,780.34,NaN
1,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,14-Nov-06,14-Nov-06,14-Nov-06,...,Oral suspension,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",358,4521.5,NaN
2,100-CI-T01,Pre-PQ Process,Côte d'Ivoire,Direct Drop,FCA,Air,Pre-PQ Process,27-Aug-06,27-Aug-06,27-Aug-06,...,Test kit,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,171,1653.78,NaN
3,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,1-Sep-06,1-Sep-06,1-Sep-06,...,Tablet,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",1855,16007.06,NaN
4,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,11-Aug-06,11-Aug-06,11-Aug-06,...,Capsule,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",7590,45450.08,NaN


In [75]:
from sklearn.pipeline import FunctionTransformer, make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

def transform_dataset(testDF: pd.DataFrame, feature_map: dict):
    # Droping the na values from the dataframe
    testDF=testDF.dropna()

    # Checking the object values in the Weight Column
    to_drop_weight= []
    for i in testDF['Weight (Kilograms)']: 
        try:
            float(i)
        except:
            to_drop_weight.append(i)
    # Dropping the object values from the Weight Column
    testDF = testDF[testDF['Weight (Kilograms)'].isin(to_drop_weight)==False]

    # Checking the object values in the Freight Column
    to_drop= []
    for i in testDF['Freight Cost (USD)']:
        try:
            float(i)
        except:
            to_drop.append(i)        
            
    # Dropping the object values from the Freight Column
    testDF = testDF[testDF['Freight Cost (USD)'].isin(to_drop)==False]

    # Splitting the scheduled Delivery date column to three respective columns storing the date, month and year
    testDF['Scheduled_Delivery_Date_D'] = [i.split('-')[0] for i in testDF['Scheduled Delivery Date']]
    testDF['Scheduled_Delivery_Date_M'] = [i.split('-')[1] for i in testDF['Scheduled Delivery Date']]
    testDF['Scheduled_Delivery_Date_Y'] = [i.split('-')[2] for i in testDF['Scheduled Delivery Date']]

    # Splitting the delivered to client date column to three respective columns storing the date, month and year
    testDF['Delivered_to_Client_Date_D'] = [i.split('-')[0] for i in testDF['Delivered to Client Date']]
    testDF['Delivered_to_Client_Date_M'] = [i.split('-')[1] for i in testDF['Delivered to Client Date']]
    testDF['Delivered_to_Client_Date_Y'] = [i.split('-')[2] for i in testDF['Delivered to Client Date']]

    # Dropping the columns: Delivered to Client Date, Scheduled Delivery Date and Delivery Recorded Date
    testDF = testDF.drop(columns=['Delivered to Client Date', 'Scheduled Delivery Date','Delivery Recorded Date'], axis= 1)

    # Changing the datatype of the columns
    testDF['Weight (Kilograms)'] = testDF['Weight (Kilograms)'].astype(float)
    testDF['Freight Cost (USD)'] = testDF['Freight Cost (USD)'].astype(float)
    testDF['Scheduled_Delivery_Date_D'] = testDF['Scheduled_Delivery_Date_D'].astype(float)
    testDF['Scheduled_Delivery_Date_Y'] = testDF['Scheduled_Delivery_Date_Y'].astype(float)
    testDF['Delivered_to_Client_Date_D'] = testDF['Delivered_to_Client_Date_D'].astype(float)
    testDF['Delivered_to_Client_Date_Y'] = testDF['Delivered_to_Client_Date_Y'].astype(float)

    
    return testDF

In [76]:
df = transform_dataset(df)

In [79]:
cat_col = df.select_dtypes(object).columns
cat_col 

In [99]:
feature_map = {}
import numpy as np
for i in cat_col:
    feature_map[i] = df[i].unique()
feature_map

{'Project Code': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], dtype=int64),
 'PQ #': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 

In [86]:
# saving the feature map as a binary file
import pickle
obj = feature_map
with open("feature_map.pkl", "wb") as file:
    pickle.dump(obj, file)

In [87]:
# loading the feature map
loaded_feature_map = None
import pickle
with open("feature_map.pkl", "rb") as file:
    loaded_feature_map = pickle.load(file)
loaded_feature_map

{'Project Code': array(['108-VN-T01', '117-ET-T01', '109-TZ-T01', '112-NG-T01',
        '104-CI-T01', '123-NG-T01', '106-HT-T01', '107-RW-T01',
        '131-NG-T01', '114-UG-T01', '119-NA-T01', '105-GY-T01',
        '130-NG-T01', '100-ZW-T01', '111-MZ-T01', '110-ZM-T01',
        '126-NG-T01', '151-NG-T01', '100-SZ-T01', '132-NG-T01',
        '113-ZW-T01', '153-NG-T01', '151-NG-T30', '108-VN-T30',
        '106-HT-T30', '111-MZ-T30', '161-ZA-T30', '107-RW-T30',
        '103-MW-T30', '104-CI-T30', '114-UG-T30', '113-ZW-T30',
        '101-CD-T30', '109-TZ-T30', '110-ZM-T30', '100-BJ-T30',
        '117-ET-T30', '900-TZ-T30', '112-NG-T30', '116-ZA-T30',
        '105-GY-T30', '901-CM-T30', '123-NG-T30', '122-HT-T30',
        '901-NA-T30', '103-ZW-T30', '105-GH-T30', '119-NA-T30',
        '200-ZW-T30', '102-ZW-T30', '900-TG-T30', '901-GH-T30',
        '103-AO-T30', '103-CM-T30', '101-AF-T30', '200-DO-T30',
        '900-CM-T30', '202-GT-T30', '100-DO-T30', '103-UG-T30',
        '900-UG-T30', '1

In [88]:
for i in cat_col:
    mapping = {}

    for j in feature_map[i]:
        index = np.where(feature_map[i] == j)[0][0]
        mapping[j] = index
        
    df[i] = df[i].map(mapping)

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4571 entries, 22 to 10316
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Project Code                  4571 non-null   int64  
 1   PQ #                          4571 non-null   int64  
 2   Country                       4571 non-null   int64  
 3   Fulfill Via                   4571 non-null   int64  
 4   Vendor INCO Term              4571 non-null   int64  
 5   Shipment Mode                 4571 non-null   int64  
 6   PQ First Sent to Client Date  4571 non-null   int64  
 7   Product Group                 4571 non-null   int64  
 8   Sub Classification            4571 non-null   int64  
 9   Vendor                        4571 non-null   int64  
 10  Item Description              4571 non-null   int64  
 11  Molecule/Test Type            4571 non-null   int64  
 12  Brand                         4571 non-null   int64  
 13  Dosage

In [90]:
num_array = {'a':np.array(['Pre-PQ Process', 'FPQ-12051', 'FPQ-9066', 'FPQ-15276',
        'FPQ-13524', 'FPQ-7175', 'FPQ-12187', 'FPQ-13976', 'FPQ-5303',
        'FPQ-15692', 'FPQ-4098', 'FPQ-6185', 'FPQ-10790', 'FPQ-14895',
        'FPQ-14372', 'FPQ-14516', 'FPQ-9365', 'FPQ-3896', 'FPQ-13395'])}

num_array

{'a': array(['Pre-PQ Process', 'FPQ-12051', 'FPQ-9066', 'FPQ-15276',
        'FPQ-13524', 'FPQ-7175', 'FPQ-12187', 'FPQ-13976', 'FPQ-5303',
        'FPQ-15692', 'FPQ-4098', 'FPQ-6185', 'FPQ-10790', 'FPQ-14895',
        'FPQ-14372', 'FPQ-14516', 'FPQ-9365', 'FPQ-3896', 'FPQ-13395'],
       dtype='<U14')}

In [91]:
index = np.where(num_array['a'] == 'Pre-PQ Process')
index[0][0]

0

In [92]:
test = ['a','b','c','d']
testdf =pd.DataFrame(np.array(test),columns=['a'])

In [93]:
testdf

,a
0,a
1,b
2,c
3,d


In [94]:
testdf['a'] = testdf['a'].map({'a':1})

In [95]:
testdf

,a
0,1.0
1,NaN
2,NaN
3,NaN


In [96]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       1 non-null      float64
dtypes: float64(1)
memory usage: 160.0 bytes


In [97]:
testdf['a'][0] = 1

C:\Users\krish\AppData\Local\Temp\ipykernel_98440\4233590120.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  testdf['a'][0] = 1


In [98]:
type(testdf['a'][0])

numpy.float64

In [104]:
import pandas as pd

In [109]:
train_df = pd.read_csv('artifacts/data_transformation/train.csv')
from sklearn.preprocessing import StandardScaler

x_train = train_df.drop(columns=['Freight Cost (USD)'], axis=1)
y_train = train_df['Freight Cost (USD)']

x_train.shape, y_train.shape 

((3428, 29), (3428,))

In [111]:
scal = StandardScaler()

x_train = scal.fit_transform(x_train)
x_train

array([[-0.17441648,  0.39859064, -0.35349024, ..., -0.79873236,
        -0.41292714, -0.66354247],
       [ 0.2966798 ,  1.57115711,  0.03193105, ..., -0.10402976,
         0.46156683, -0.66354247],
       [ 0.61074399,  1.42062493,  0.99548429, ..., -1.26186743,
        -0.99592312,  0.71501728],
       ...,
       [-0.33144858,  1.05617859, -0.73891154, ...,  0.47488908,
        -0.41292714,  1.1745372 ],
       [ 0.61074399,  1.26217   ,  0.99548429, ..., -0.10402976,
         1.3360608 , -0.20402255],
       [-1.11660905, -1.15426765, -0.73891154, ...,  0.47488908,
        -0.99592312, -1.58258231]])

In [112]:
with open('scal.pkl', 'wb') as file:
    pickle.dump(scal, file)


In [113]:
obj = None 
with open('scal.pkl', 'rb') as file:
    obj = pickle.load(file)

obj 

StandardScaler()

In [117]:
test_df = pd.read_csv('artifacts/data_transformation/test.csv')

In [118]:
x_test = test_df.drop(columns=['Freight Cost (USD)'], axis=1)
y_test = test_df['Freight Cost (USD)']

x_test.shape, y_test.shape 

((1143, 29), (1143,))

In [119]:
x_test

,Project Code,PQ #,Country,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,Product Group,Sub Classification,Vendor,...,Unit Price,Manufacturing Site,Weight (Kilograms),Line Item Insurance (USD),Scheduled_Delivery_Date_D,Scheduled_Delivery_Date_M,Scheduled_Delivery_Date_Y,Delivered_to_Client_Date_D,Delivered_to_Client_Date_M,Delivered_to_Client_Date_Y
0,46,183,19,0,0,0,168,0,0,0,...,0.01,0,2323.0,8.89,12.0,4,14.0,12.0,4,14.0
1,4,0,4,0,1,0,0,0,1,1,...,0.34,1,2523.0,824.89,30.0,4,9.0,30.0,4,9.0
2,17,0,3,1,3,0,0,0,1,16,...,0.88,14,1481.0,878.10,29.0,8,9.0,29.0,7,9.0
3,33,533,2,1,3,1,400,0,1,16,...,0.16,1,8820.0,21.95,31.0,3,14.0,19.0,10,14.0
4,25,419,11,1,3,0,330,0,1,16,...,0.05,4,643.0,60.51,30.0,2,10.0,13.0,11,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,25,144,11,0,1,0,134,0,1,1,...,0.11,1,87.0,5.22,8.0,0,13.0,8.0,0,13.0
1139,25,417,11,1,3,1,328,0,1,16,...,0.19,0,28237.0,5043.71,25.0,2,10.0,1.0,11,10.0
1140,29,280,4,0,0,0,242,0,1,4,...,0.47,4,316.0,96.10,9.0,9,12.0,9.0,8,12.0
1141,1,0,1,1,3,0,1,0,1,16,...,0.05,10,2190.0,125.05,8.0,11,9.0,8.0,11,9.0


In [116]:
x_test = scal.transform(x_test)
x_test

array([[ 1.55293656, -0.42933635,  2.34445882, ..., -0.45138106,
        -0.41292714,  1.1745372 ],
       [-1.74473743, -1.15426765, -0.54620089, ...,  1.63272676,
        -0.41292714, -1.12306239],
       [-0.72402881, -1.15426765, -0.73891154, ...,  1.51694299,
         0.46156683, -1.12306239],
       ...,
       [ 0.21816375, -0.04508315, -0.54620089, ..., -0.79873236,
         0.75306482,  0.25549737],
       [-1.98028557, -1.15426765, -1.12433283, ..., -0.91451613,
         1.62755879, -1.12306239],
       [-0.17441648,  0.39859064, -0.35349024, ..., -1.72500251,
        -0.41292714, -0.66354247]])

In [120]:
x_test = obj.transform(x_test)
x_test

array([[ 1.55293656, -0.42933635,  2.34445882, ..., -0.45138106,
        -0.41292714,  1.1745372 ],
       [-1.74473743, -1.15426765, -0.54620089, ...,  1.63272676,
        -0.41292714, -1.12306239],
       [-0.72402881, -1.15426765, -0.73891154, ...,  1.51694299,
         0.46156683, -1.12306239],
       ...,
       [ 0.21816375, -0.04508315, -0.54620089, ..., -0.79873236,
         0.75306482,  0.25549737],
       [-1.98028557, -1.15426765, -1.12433283, ..., -0.91451613,
         1.62755879, -1.12306239],
       [-0.17441648,  0.39859064, -0.35349024, ..., -1.72500251,
        -0.41292714, -0.66354247]])

In [157]:
a = {
    'zsdf':10,
    'qwer':5,
    'asdfasd':2
}
# for i in a.items():
#     print(i[0])

# c = max(a.values())
# list(a.values()).index(c)

list(a.items())[0][0]

'zsdf'

In [167]:
a = {'LinearRegression': 0.3077374722825449, 'Lasso': 0.3077311569923059, 'Ridge': 0.30773270424003385}
c = max(a.values())
print(list(a.items())[list(a.values()).index(c)][0])

LinearRegression
